# NLP Comparative Analysis Toolkit (NLP-CAT) 2.1: A Comprehensive Study of Text Classification Paradigms

**Author:** Daniel Wanjala Machimbo  
**Institution:** The Cooperative University of Kenya  
**Date:** October 2025  
**Python Version:** 3.11.13  

---

## Reproducibility Badge

| Criterion | Status |
|-----------|---------|
| **Code Available** | ✅ Yes - Complete implementation |
| **Data Public** | ✅ Yes - AG News, 20 Newsgroups, IMDb |
| **Seeds Fixed** | ✅ Yes - [42, 101, 2023, 7, 999] |
| **Environment Specified** | ✅ Yes - requirements.txt provided |
| **Statistical Tests** | ✅ Yes - Wilcoxon, Cohen's d, Bootstrap CI |

---

## What This Notebook Will Produce

### Artifacts Generated:
- **Models**: `artifacts/classical/`, `artifacts/bilstm/`, `artifacts/bert/`, `artifacts/hybrid/`
- **Results**: `results/summary.csv`, `results/statistics.json`
- **Applications**: `app_streamlit.py` (React-level dashboard)
- **Utilities**: `train.py` (CLI wrapper), `requirements.txt`
- **Data**: `data/manifest.json` (dataset checksums)

### Commands to Execute:
```bash
# Run notebook end-to-end (non-interactive)
papermill NLP_CAT_comparative_study.ipynb output.ipynb -p run_full true

# Launch interactive dashboard
streamlit run app_streamlit.py --server.port 8501

# Train single model configuration
python train.py --dataset ag_news --model bert --n_samples 1000 --seed 42
```

### Expected Runtime:
- **Classical Models**: ~5-10 minutes per dataset
- **BiLSTM**: ~15-30 minutes per dataset  
- **BERT**: ~45-90 minutes per dataset (GPU), 4-8 hours (CPU)
- **Full Experiment Suite**: ~6-12 hours (GPU), ~24-48 hours (CPU)

# 1. Abstract

This comprehensive study presents a rigorous empirical comparison of four distinct text classification paradigms across three canonical datasets. We systematically evaluate classical machine learning approaches (Multinomial Naïve Bayes and Linear Support Vector Machines with TF-IDF features), recurrent neural networks (Bidirectional LSTM with GloVe embeddings), and modern transformer architectures (BERT-base-uncased) on AG News (4-class news categorization), 20 Newsgroups (20-class discussion forum classification), and IMDb movie reviews (binary sentiment analysis).

Our experimental protocol examines model performance across multiple labeled-sample regimes (1K, 5K, 10K, and full datasets) using five independent random seeds to ensure statistical robustness. We employ comprehensive evaluation metrics including accuracy, macro-F1 score, negative log-likelihood, Expected Calibration Error (ECE), per-class performance metrics, inference latency, model size, and computational complexity proxies.

**Key Findings** (to be populated after experimentation): Classical methods demonstrate superior computational efficiency and competitive performance on smaller datasets, while transformer models achieve state-of-the-art accuracy at significant computational cost. Our calibration analysis reveals systematic overconfidence in neural models, addressable through temperature scaling. Statistical testing using paired Wilcoxon signed-rank tests and Cohen's d effect sizes provides rigorous significance assessment.

This work contributes a reproducible experimental framework with complete statistical analysis, model persistence, and an interactive Streamlit dashboard for real-time model comparison and interpretation. All code, data preprocessing pipelines, and trained models are made available for scientific reproducibility.

# 2. Problem Statement & Objectives

## Problem Statement

Text classification represents a fundamental task in natural language processing with broad applications across information retrieval, content moderation, sentiment analysis, and automated document processing. While the field has witnessed rapid advancement from classical statistical methods to modern transformer architectures, practitioners face critical decisions regarding model selection under varying computational constraints, dataset sizes, and performance requirements.

The central research question driving this investigation is: **How do classical machine learning approaches, recurrent neural networks, and transformer models compare across multiple dimensions of performance when evaluated systematically on diverse text classification tasks?**

## Research Objectives

### Primary Objectives:
1. **Comparative Performance Analysis**: Quantify accuracy, calibration, and efficiency trade-offs across four model families
2. **Sample Efficiency Assessment**: Characterize learning curves across multiple labeled-sample regimes
3. **Statistical Robustness**: Establish significance of performance differences using rigorous statistical testing
4. **Practical Deployment Guidance**: Provide actionable insights for model selection in resource-constrained environments

## Formal Hypotheses

**H1 (Performance Hierarchy)**: Transformer models (BERT) will achieve superior classification accuracy compared to classical and recurrent approaches, with the performance ranking: BERT > BiLSTM > LinearSVM > MultinomialNB.

**H2 (Sample Efficiency)**: Classical methods will demonstrate superior performance in low-data regimes (n ≤ 1000), while transformer models will show increasing relative advantage as sample size increases.

**H3 (Efficiency-Accuracy Pareto Frontier)**: A clear Pareto frontier will emerge in the accuracy-computational cost space, with classical methods occupying the efficient low-cost region and transformers the high-accuracy high-cost region.

**H4 (Calibration Hypothesis)**: Neural models (BiLSTM, BERT) will exhibit systematic overconfidence compared to classical approaches, measurable through Expected Calibration Error (ECE) metrics.

## Scientific Significance

This study addresses a critical gap in the literature by providing a comprehensive, statistically rigorous comparison across multiple evaluation dimensions. Unlike previous works that focus on accuracy alone, we incorporate calibration assessment, computational efficiency analysis, and robust statistical testing to provide practitioners with actionable insights for model selection in production environments.

In [ ]:
# Environment Setup and Reproducibility Configuration
# This cell establishes the complete computational environment for our experiments

import os
import sys
import warnings
import time
import json
import hashlib
import subprocess
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Tuple, Optional, Union, Any
from dataclasses import dataclass, asdict

# Suppress warnings for cleaner output during experimentation
warnings.filterwarnings('ignore')

# Set random seeds for complete reproducibility
RANDOM_SEEDS = [42, 101, 2023, 7, 999]
DEFAULT_SEED = RANDOM_SEEDS[0]

import random
import numpy as np
random.seed(DEFAULT_SEED)
np.random.seed(DEFAULT_SEED)

# Create necessary directories
os.makedirs('artifacts', exist_ok=True)
os.makedirs('artifacts/classical', exist_ok=True)
os.makedirs('artifacts/bilstm', exist_ok=True)
os.makedirs('artifacts/bert', exist_ok=True)
os.makedirs('artifacts/hybrid', exist_ok=True)
os.makedirs('results', exist_ok=True)
os.makedirs('data', exist_ok=True)

print("✓ Directory structure created successfully")
print(f"✓ Random seeds configured: {RANDOM_SEEDS}")
print(f"✓ Default seed set to: {DEFAULT_SEED}")
print(f"✓ Python version: {sys.version}")
print(f"✓ Working directory: {os.getcwd()}")

In [ ]:
# Import Core Libraries for Data Processing and Machine Learning
print("Importing core scientific computing libraries...")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import wilcoxon
import sklearn
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import (accuracy_score, f1_score, precision_recall_fscore_support, 
                           confusion_matrix, classification_report, log_loss)
from sklearn.calibration import calibration_curve, CalibratedClassifierCV
import joblib

print("✓ Sklearn and scipy libraries imported")

# NLP-specific libraries
print("Importing NLP libraries...")
import nltk
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Download required NLTK data
try:
    nltk.download('punkt', quiet=True)
    nltk.download('stopwords', quiet=True) 
    nltk.download('wordnet', quiet=True)
    nltk.download('omw-1.4', quiet=True)
    print("✓ NLTK data downloaded successfully")
except Exception as e:
    print(f"Warning: NLTK download issue: {e}")

print("✓ NLP libraries imported")

In [ ]:
# Import Deep Learning Libraries (PyTorch and Transformers)
print("Importing deep learning libraries...")

try:
    import torch
    import torch.nn as nn
    import torch.nn.functional as F
    from torch.utils.data import Dataset, DataLoader, TensorDataset
    from torch.optim import Adam, AdamW
    from torch.optim.lr_scheduler import ReduceLROnPlateau
    
    # Set PyTorch for reproducibility
    torch.manual_seed(DEFAULT_SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    # Check for GPU availability
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"✓ PyTorch imported successfully - Device: {device}")
    if torch.cuda.is_available():
        print(f"✓ GPU detected: {torch.cuda.get_device_name(0)}")
        print(f"✓ GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    
except ImportError as e:
    print(f"Warning: PyTorch not available - {e}")
    device = 'cpu'

try:
    import transformers
    from transformers import (AutoTokenizer, AutoModelForSequenceClassification, 
                            Trainer, TrainingArguments, EarlyStoppingCallback,
                            BertTokenizer, BertForSequenceClassification)
    
    # Set transformers logging level to reduce noise
    transformers.logging.set_verbosity_error()
    print("✓ Transformers library imported successfully")
    
except ImportError as e:
    print(f"Warning: Transformers not available - {e}")

try:
    import datasets
    from datasets import load_dataset, Dataset as HFDataset
    print("✓ HuggingFace datasets imported successfully")
except ImportError as e:
    print(f"Warning: HuggingFace datasets not available - {e}")

# Import progress tracking
from tqdm.auto import tqdm
tqdm.pandas()

print("✓ All deep learning libraries configured")

# 3. Datasets & Study Area

## Dataset Selection Rationale

Our experimental design employs three carefully selected datasets that represent distinct text classification challenges across different domains, text lengths, and class distributions:

1. **AG News** (Short-form news categorization): 4-class classification with concise, structured text
2. **20 Newsgroups** (Medium-form discussion classification): 20-class classification with conversational text
3. **IMDb Movie Reviews** (Long-form sentiment analysis): Binary sentiment classification with extended reviews

This selection ensures our findings generalize across varying textual characteristics and classification complexity levels.

## Ethical Considerations and Data Usage

All datasets employed in this study are publicly available, extensively used in academic research, and do not contain personally identifiable information (PII). We acknowledge potential demographic biases present in these datasets and will address fairness considerations in our analysis. Our use complies with respective dataset licenses and academic fair use principles.

## Dataset Loading Infrastructure

The following implementation provides robust, reproducible dataset loading with comprehensive error handling, caching mechanisms, and metadata tracking. Each dataset is loaded programmatically with fallback options and complete provenance tracking.

In [ ]:
# Dataset Loading Functions with Comprehensive Error Handling
@dataclass
class DatasetInfo:
    """Metadata container for dataset information tracking"""
    name: str
    source: str
    license: str
    classes: int
    train_size: int
    test_size: int
    avg_length: float
    md5_hash: str
    load_time: float

def compute_md5_hash(texts: List[str], labels: List[int]) -> str:
    """Compute MD5 hash of dataset for integrity verification"""
    content = ''.join(texts) + ''.join(map(str, labels))
    return hashlib.md5(content.encode()).hexdigest()

def load_ag_news_dataset() -> Tuple[List[str], List[int], List[str], List[int], DatasetInfo]:
    """
    Load AG News dataset using HuggingFace datasets with fallback options.
    
    Source: https://huggingface.co/datasets/ag_news
    License: Apache License 2.0
    Classes: 4 (World, Sports, Business, Sci/Tech)
    """
    print("Loading AG News dataset...")
    start_time = time.time()
    
    try:
        # Primary method: HuggingFace datasets
        dataset = load_dataset("ag_news", cache_dir="data/cache")
        
        train_texts = [item['text'] for item in dataset['train']]
        train_labels = [item['label'] for item in dataset['train']]
        test_texts = [item['text'] for item in dataset['test']]
        test_labels = [item['label'] for item in dataset['test']]
        
        print(f"✓ AG News loaded via HuggingFace datasets")
        
    except Exception as e:
        print(f"HuggingFace loading failed: {e}")
        print("Attempting torchtext fallback...")
        
        try:
            # Fallback method: torchtext (if available)
            import torchtext
            from torchtext.datasets import AG_NEWS
            
            train_iter, test_iter = AG_NEWS(root='data', split=('train', 'test'))
            
            train_data = list(train_iter)
            test_data = list(test_iter)
            
            train_labels = [int(label) - 1 for label, text in train_data]  # Convert to 0-indexed
            train_texts = [text for label, text in train_data]
            test_labels = [int(label) - 1 for label, text in test_data]
            test_texts = [text for label, text in test_data]
            
            print(f"✓ AG News loaded via torchtext fallback")
            
        except Exception as e2:
            print(f"Torchtext fallback failed: {e2}")
            raise RuntimeError("Failed to load AG News dataset with both methods")
    
    # Calculate metadata
    avg_length = np.mean([len(text.split()) for text in train_texts + test_texts])
    md5_hash = compute_md5_hash(train_texts + test_texts, train_labels + test_labels)
    load_time = time.time() - start_time
    
    dataset_info = DatasetInfo(
        name="AG_News",
        source="https://huggingface.co/datasets/ag_news",
        license="Apache License 2.0",
        classes=4,
        train_size=len(train_texts),
        test_size=len(test_texts),
        avg_length=avg_length,
        md5_hash=md5_hash,
        load_time=load_time
    )
    
    print(f"✓ AG News: {dataset_info.train_size} train, {dataset_info.test_size} test samples")
    print(f"✓ Average text length: {avg_length:.1f} words")
    
    return train_texts, train_labels, test_texts, test_labels, dataset_info

# Load AG News dataset
ag_train_texts, ag_train_labels, ag_test_texts, ag_test_labels, ag_info = load_ag_news_dataset()

In [ ]:
def load_20newsgroups_dataset() -> Tuple[List[str], List[int], List[str], List[int], DatasetInfo]:
    """
    Load 20 Newsgroups dataset using sklearn with header/footer/quote removal.
    
    Source: sklearn.datasets.fetch_20newsgroups
    License: Public Domain
    Classes: 20 (various newsgroup categories)
    """
    print("Loading 20 Newsgroups dataset...")
    start_time = time.time()
    
    # Load with preprocessing to remove headers, footers, and quotes
    # This is crucial for fair evaluation as it removes metadata that could be used for cheating
    train_data = fetch_20newsgroups(
        subset='train', 
        remove=('headers', 'footers', 'quotes'),
        shuffle=True, 
        random_state=DEFAULT_SEED,
        data_home='data'
    )
    
    test_data = fetch_20newsgroups(
        subset='test', 
        remove=('headers', 'footers', 'quotes'),
        shuffle=True, 
        random_state=DEFAULT_SEED,
        data_home='data'
    )
    
    train_texts = train_data.data
    train_labels = train_data.target.tolist()
    test_texts = test_data.data
    test_labels = test_data.target.tolist()
    
    # Calculate metadata
    avg_length = np.mean([len(text.split()) for text in train_texts + test_texts])
    md5_hash = compute_md5_hash(train_texts + test_texts, train_labels + test_labels)
    load_time = time.time() - start_time
    
    dataset_info = DatasetInfo(
        name="20_Newsgroups",
        source="sklearn.datasets.fetch_20newsgroups",
        license="Public Domain",
        classes=20,
        train_size=len(train_texts),
        test_size=len(test_texts),
        avg_length=avg_length,
        md5_hash=md5_hash,
        load_time=load_time
    )
    
    print(f"✓ 20 Newsgroups: {dataset_info.train_size} train, {dataset_info.test_size} test samples")
    print(f"✓ Average text length: {avg_length:.1f} words")
    print(f"✓ Target names: {train_data.target_names[:5]}... (showing first 5)")
    
    return train_texts, train_labels, test_texts, test_labels, dataset_info

# Load 20 Newsgroups dataset
ng_train_texts, ng_train_labels, ng_test_texts, ng_test_labels, ng_info = load_20newsgroups_dataset()

In [ ]:
def load_imdb_dataset() -> Tuple[List[str], List[int], List[str], List[int], DatasetInfo]:
    """
    Load IMDb movie reviews dataset using HuggingFace datasets.
    
    Source: https://huggingface.co/datasets/imdb
    License: Apache License 2.0
    Classes: 2 (positive, negative sentiment)
    """
    print("Loading IMDb dataset...")
    start_time = time.time()
    
    try:
        # Primary method: HuggingFace datasets
        dataset = load_dataset("imdb", cache_dir="data/cache")
        
        train_texts = [item['text'] for item in dataset['train']]
        train_labels = [item['label'] for item in dataset['train']]
        test_texts = [item['text'] for item in dataset['test']]
        test_labels = [item['label'] for item in dataset['test']]
        
        print(f"✓ IMDb loaded via HuggingFace datasets")
        
    except Exception as e:
        print(f"HuggingFace loading failed: {e}")
        print("Attempting tensorflow_datasets fallback...")
        
        try:
            # Fallback method: tensorflow_datasets (if available)
            import tensorflow_datasets as tfds
            
            ds_train = tfds.load('imdb_reviews', split='train', as_supervised=True, 
                               data_dir='data/tfds_cache')
            ds_test = tfds.load('imdb_reviews', split='test', as_supervised=True,
                              data_dir='data/tfds_cache')
            
            train_texts = []
            train_labels = []
            for text, label in ds_train:
                train_texts.append(text.numpy().decode('utf-8'))
                train_labels.append(int(label.numpy()))
                
            test_texts = []
            test_labels = []
            for text, label in ds_test:
                test_texts.append(text.numpy().decode('utf-8'))
                test_labels.append(int(label.numpy()))
            
            print(f"✓ IMDb loaded via tensorflow_datasets fallback")
            
        except Exception as e2:
            print(f"TensorFlow datasets fallback failed: {e2}")
            raise RuntimeError("Failed to load IMDb dataset with both methods")
    
    # Calculate metadata
    avg_length = np.mean([len(text.split()) for text in train_texts + test_texts])
    md5_hash = compute_md5_hash(train_texts + test_texts, train_labels + test_labels)
    load_time = time.time() - start_time
    
    dataset_info = DatasetInfo(
        name="IMDb",
        source="https://huggingface.co/datasets/imdb",
        license="Apache License 2.0", 
        classes=2,
        train_size=len(train_texts),
        test_size=len(test_texts),
        avg_length=avg_length,
        md5_hash=md5_hash,
        load_time=load_time
    )
    
    print(f"✓ IMDb: {dataset_info.train_size} train, {dataset_info.test_size} test samples")
    print(f"✓ Average text length: {avg_length:.1f} words")
    
    return train_texts, train_labels, test_texts, test_labels, dataset_info

# Load IMDb dataset
imdb_train_texts, imdb_train_labels, imdb_test_texts, imdb_test_labels, imdb_info = load_imdb_dataset()

In [ ]:
# Dataset Metadata Tracking and Manifest Generation
def save_dataset_manifest(datasets_info: List[DatasetInfo]) -> None:
    """Save dataset metadata to JSON manifest for reproducibility tracking"""
    manifest = {
        'generated_at': datetime.now().isoformat(),
        'python_version': sys.version,
        'random_seed': DEFAULT_SEED,
        'datasets': {info.name: asdict(info) for info in datasets_info}
    }
    
    with open('data/manifest.json', 'w') as f:
        json.dump(manifest, f, indent=2)
    
    print(f"✓ Dataset manifest saved to data/manifest.json")

# Collect all dataset information
all_datasets_info = [ag_info, ng_info, imdb_info]

# Save manifest
save_dataset_manifest(all_datasets_info)

# Display dataset summary table
print("\n" + "="*80)
print("DATASET SUMMARY STATISTICS")
print("="*80)

summary_df = pd.DataFrame([asdict(info) for info in all_datasets_info])
summary_df = summary_df[['name', 'classes', 'train_size', 'test_size', 'avg_length', 'load_time']]
summary_df['avg_length'] = summary_df['avg_length'].round(1)
summary_df['load_time'] = summary_df['load_time'].round(2)

print(summary_df.to_string(index=False))
print("="*80)

In [ ]:
# Comprehensive Exploratory Data Analysis (EDA)
def perform_dataset_eda(texts: List[str], labels: List[int], dataset_name: str, 
                       label_names: Optional[List[str]] = None) -> Dict[str, Any]:
    """
    Perform comprehensive exploratory data analysis on a text dataset.
    
    Returns statistical summaries and generates publication-quality visualizations.
    """
    print(f"\nAnalyzing {dataset_name} dataset...")
    
    # Basic statistics
    n_samples = len(texts)
    n_classes = len(set(labels))
    
    # Text length analysis
    text_lengths = [len(text.split()) for text in texts]
    length_stats = {
        'mean': np.mean(text_lengths),
        'std': np.std(text_lengths),
        'min': np.min(text_lengths),
        'max': np.max(text_lengths),
        'median': np.median(text_lengths),
        'q25': np.percentile(text_lengths, 25),
        'q75': np.percentile(text_lengths, 75)
    }
    
    # Class distribution analysis
    class_counts = pd.Series(labels).value_counts().sort_index()
    class_distribution = {
        'counts': class_counts.to_dict(),
        'proportions': (class_counts / class_counts.sum()).to_dict(),
        'imbalance_ratio': class_counts.max() / class_counts.min()
    }
    
    # Character-level statistics
    char_lengths = [len(text) for text in texts]
    char_stats = {
        'mean_chars': np.mean(char_lengths),
        'std_chars': np.std(char_lengths)
    }
    
    # Vocabulary analysis (approximate)
    all_words = []
    for text in texts[:1000]:  # Sample for efficiency
        all_words.extend(text.lower().split())
    
    vocab_stats = {
        'unique_words_sample': len(set(all_words)),
        'total_words_sample': len(all_words),
        'avg_word_length': np.mean([len(word) for word in all_words])
    }
    
    eda_results = {
        'dataset_name': dataset_name,
        'n_samples': n_samples,
        'n_classes': n_classes,
        'length_stats': length_stats,
        'char_stats': char_stats,
        'class_distribution': class_distribution,
        'vocab_stats': vocab_stats
    }
    
    # Create publication-quality visualizations
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle(f'{dataset_name} Dataset Analysis', fontsize=16, fontweight='bold')
    
    # 1. Text length distribution
    axes[0, 0].hist(text_lengths, bins=50, alpha=0.7, color='skyblue', edgecolor='black')
    axes[0, 0].axvline(length_stats['mean'], color='red', linestyle='--', 
                       label=f'Mean: {length_stats["mean"]:.1f}')
    axes[0, 0].axvline(length_stats['median'], color='green', linestyle='--', 
                       label=f'Median: {length_stats["median"]:.1f}')
    axes[0, 0].set_xlabel('Text Length (words)')
    axes[0, 0].set_ylabel('Frequency')
    axes[0, 0].set_title('Text Length Distribution')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. Class distribution
    class_labels = label_names if label_names else [f'Class {i}' for i in range(n_classes)]
    if len(class_labels) <= 10:  # Full labels for manageable number of classes
        axes[0, 1].bar(range(len(class_counts)), class_counts.values, 
                       color='lightcoral', alpha=0.7, edgecolor='black')
        axes[0, 1].set_xticks(range(len(class_counts)))
        axes[0, 1].set_xticklabels([class_labels[i] for i in class_counts.index], 
                                   rotation=45, ha='right')
    else:  # Simplified for many classes
        axes[0, 1].bar(range(len(class_counts)), class_counts.values, 
                       color='lightcoral', alpha=0.7, edgecolor='black')
        axes[0, 1].set_xlabel('Class Index')
    
    axes[0, 1].set_ylabel('Sample Count')
    axes[0, 1].set_title(f'Class Distribution (Imbalance Ratio: {class_distribution["imbalance_ratio"]:.2f})')
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. Length vs Class boxplot (for reasonable number of classes)
    if n_classes <= 10:
        length_by_class = [[] for _ in range(n_classes)]
        for text, label in zip(texts, labels):
            length_by_class[label].append(len(text.split()))
        
        axes[1, 0].boxplot(length_by_class, labels=[f'C{i}' for i in range(n_classes)])
        axes[1, 0].set_xlabel('Class')
        axes[1, 0].set_ylabel('Text Length (words)')
        axes[1, 0].set_title('Text Length Distribution by Class')
        axes[1, 0].grid(True, alpha=0.3)
    else:
        # Alternative visualization for many classes
        axes[1, 0].scatter(labels[:1000], [len(texts[i].split()) for i in range(1000)], 
                          alpha=0.5, s=1)
        axes[1, 0].set_xlabel('Class Index')
        axes[1, 0].set_ylabel('Text Length (words)')
        axes[1, 0].set_title('Text Length vs Class (Sample)')
        axes[1, 0].grid(True, alpha=0.3)
    
    # 4. Summary statistics table
    axes[1, 1].axis('off')
    stats_text = f'''
    Dataset Statistics:
    
    Samples: {n_samples:,}
    Classes: {n_classes}
    
    Text Length (words):
      Mean: {length_stats["mean"]:.1f} ± {length_stats["std"]:.1f}
      Median: {length_stats["median"]:.1f}
      Range: [{length_stats["min"]}, {length_stats["max"]}]
      
    Characters per text:
      Mean: {char_stats["mean_chars"]:.0f} ± {char_stats["std_chars"]:.0f}
      
    Class Balance:
      Most frequent: {class_counts.max():,} samples
      Least frequent: {class_counts.min():,} samples
      Imbalance ratio: {class_distribution["imbalance_ratio"]:.2f}
    '''
    axes[1, 1].text(0.1, 0.9, stats_text, transform=axes[1, 1].transAxes, 
                     fontsize=10, verticalalignment='top', fontfamily='monospace',
                     bbox=dict(boxstyle='round', facecolor='lightgray', alpha=0.8))
    
    plt.tight_layout()
    plt.savefig(f'results/{dataset_name.lower()}_eda.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    return eda_results

# Perform EDA for all datasets
print("Conducting comprehensive exploratory data analysis...")

# AG News EDA
ag_class_names = ['World', 'Sports', 'Business', 'Sci/Tech']
ag_eda = perform_dataset_eda(ag_train_texts, ag_train_labels, 'AG_News', ag_class_names)

In [ ]:
# Continue EDA for remaining datasets and display sample texts
print("\n" + "="*60)

# 20 Newsgroups EDA  
ng_class_names = ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 
                  'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x',
                  'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball',
                  'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med',
                  'sci.space', 'soc.religion.christian', 'talk.politics.guns',
                  'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']
ng_eda = perform_dataset_eda(ng_train_texts, ng_train_labels, '20_Newsgroups', ng_class_names)

print("\n" + "="*60)

# IMDb EDA
imdb_class_names = ['Negative', 'Positive'] 
imdb_eda = perform_dataset_eda(imdb_train_texts, imdb_train_labels, 'IMDb', imdb_class_names)

# Display sample texts from each dataset for qualitative understanding
print("\n" + "="*80)
print("SAMPLE TEXTS FROM EACH DATASET")
print("="*80)

def display_sample_texts(texts: List[str], labels: List[int], dataset_name: str, 
                        class_names: List[str], n_samples: int = 2) -> None:
    """Display sample texts from each class for qualitative analysis"""
    print(f"\n{dataset_name} Sample Texts:")
    print("-" * 50)
    
    unique_labels = sorted(set(labels))
    for label in unique_labels[:min(len(unique_labels), 4)]:  # Show up to 4 classes
        label_indices = [i for i, l in enumerate(labels) if l == label]
        sample_indices = np.random.choice(label_indices, min(n_samples, len(label_indices)), 
                                        replace=False)
        
        print(f"\nClass: {class_names[label] if label < len(class_names) else f'Class_{label}'}")
        for i, idx in enumerate(sample_indices):
            text_preview = texts[idx][:200] + "..." if len(texts[idx]) > 200 else texts[idx]
            print(f"  Sample {i+1}: {text_preview}")
            print()

# Set random seed for consistent sampling
np.random.seed(DEFAULT_SEED)

display_sample_texts(ag_train_texts, ag_train_labels, "AG News", ag_class_names)
display_sample_texts(ng_train_texts, ng_train_labels, "20 Newsgroups", ng_class_names)  
display_sample_texts(imdb_train_texts, imdb_train_labels, "IMDb", imdb_class_names)

print("="*80)

# 4. Preprocessing Pipeline

## Text Preprocessing Philosophy and Implementation

Text preprocessing represents a critical yet often underappreciated component of NLP pipeline design. Our approach implements a flexible, modular preprocessing framework that enables systematic ablation studies while maintaining reproducibility across different model architectures.

### Preprocessing Considerations:

1. **Normalization**: Converting text to consistent case and removing extraneous characters
2. **Tokenization**: Word-level vs. subword tokenization strategies  
3. **Stop Word Removal**: Impact on different classification paradigms
4. **Lemmatization**: Computational cost vs. potential benefit analysis
5. **Feature Engineering**: N-gram extraction and TF-IDF parameterization

### Design Principles:

- **Modularity**: Each preprocessing step can be toggled independently
- **Consistency**: Identical preprocessing for fair model comparison
- **Efficiency**: Optimized implementations with caching for large datasets
- **Reproducibility**: Deterministic operations with fixed parameters

The following implementation provides comprehensive preprocessing utilities with extensive parameter control, enabling both classical feature extraction and modern tokenizer compatibility.

In [ ]:
# Comprehensive Text Preprocessing Pipeline
import re
import string
from typing import Callable

# Initialize preprocessing components
try:
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    print("✓ NLTK preprocessing components initialized")
except Exception as e:
    print(f"Warning: NLTK initialization issue: {e}")
    stop_words = set()
    lemmatizer = None

@dataclass  
class PreprocessingConfig:
    """Configuration class for text preprocessing parameters"""
    lowercase: bool = True
    remove_punctuation: bool = True
    remove_digits: bool = False
    remove_stopwords: bool = True
    lemmatize: bool = False
    min_token_length: int = 2
    max_token_length: int = 50

def clean_text(text: str, config: PreprocessingConfig = PreprocessingConfig()) -> str:
    """
    Comprehensive text cleaning function with configurable options.
    
    Args:
        text: Input text string
        config: PreprocessingConfig object with cleaning parameters
        
    Returns:
        Cleaned text string
    """
    if not isinstance(text, str):
        text = str(text)
    
    # Basic cleaning - remove excessive whitespace and normalize
    text = re.sub(r'\s+', ' ', text.strip())
    
    # Convert to lowercase if specified
    if config.lowercase:
        text = text.lower()
    
    # Remove URLs, email addresses, and mentions
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'\S+@\S+', '', text)
    text = re.sub(r'@\w+', '', text)
    
    # Remove digits if specified
    if config.remove_digits:
        text = re.sub(r'\d+', '', text)
    
    # Remove punctuation if specified (preserve word boundaries)
    if config.remove_punctuation:
        text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Normalize whitespace again after punctuation removal
    text = re.sub(r'\s+', ' ', text.strip())
    
    return text

def tokenize_text(text: str, config: PreprocessingConfig = PreprocessingConfig()) -> List[str]:
    """
    Advanced tokenization with filtering and optional lemmatization.
    
    Args:
        text: Input text string
        config: PreprocessingConfig object with tokenization parameters
        
    Returns:
        List of processed tokens
    """
    # Clean text first
    text = clean_text(text, config)
    
    # Tokenize using NLTK word_tokenize (handles contractions better than split())
    try:
        tokens = word_tokenize(text)
    except:
        # Fallback to simple split if NLTK fails
        tokens = text.split()
    
    # Filter tokens by length
    tokens = [token for token in tokens 
              if config.min_token_length <= len(token) <= config.max_token_length]
    
    # Remove stopwords if specified
    if config.remove_stopwords and stop_words:
        tokens = [token for token in tokens if token.lower() not in stop_words]
    
    # Apply lemmatization if specified and available
    if config.lemmatize and lemmatizer:
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    return tokens

def create_tfidf_vectorizer(ngram_range: Tuple[int, int] = (1, 1),
                           max_features: int = 10000,
                           use_idf: bool = True,
                           preprocessing_config: PreprocessingConfig = PreprocessingConfig()) -> TfidfVectorizer:
    """
    Create configured TF-IDF vectorizer with custom preprocessing.
    
    Args:
        ngram_range: Tuple of (min_n, max_n) for n-gram extraction
        max_features: Maximum number of features to extract
        use_idf: Whether to use IDF weighting
        preprocessing_config: Text preprocessing configuration
        
    Returns:
        Configured TfidfVectorizer instance
    """
    
    def custom_preprocessor(text: str) -> str:
        """Custom preprocessor function for TfidfVectorizer"""
        return clean_text(text, preprocessing_config)
    
    def custom_tokenizer(text: str) -> List[str]:
        """Custom tokenizer function for TfidfVectorizer"""
        return tokenize_text(text, preprocessing_config)
    
    vectorizer = TfidfVectorizer(
        preprocessor=custom_preprocessor,
        tokenizer=custom_tokenizer,
        ngram_range=ngram_range,
        max_features=max_features,
        use_idf=use_idf,
        lowercase=False,  # Already handled in custom preprocessor
        stop_words=None,  # Already handled in custom tokenizer
        dtype=np.float32  # Use float32 for memory efficiency
    )
    
    return vectorizer

# Test preprocessing pipeline with examples
def test_preprocessing_pipeline():
    """Test and demonstrate preprocessing pipeline functionality"""
    
    test_texts = [
        "Hello World! This is a TEST with numbers 123 and punctuation...",
        "Check out this URL: https://example.com and email test@email.com",
        "Multiple    spaces   and\t\ttabs should be normalized!!!",
        "Contractions like don't, won't, and I'm should be handled properly."
    ]
    
    # Test different configurations
    configs = {
        'minimal': PreprocessingConfig(lowercase=True, remove_punctuation=False, 
                                     remove_stopwords=False, lemmatize=False),
        'standard': PreprocessingConfig(lowercase=True, remove_punctuation=True, 
                                      remove_stopwords=True, lemmatize=False),
        'aggressive': PreprocessingConfig(lowercase=True, remove_punctuation=True, 
                                        remove_stopwords=True, remove_digits=True, 
                                        lemmatize=True)
    }
    
    print("PREPROCESSING PIPELINE TESTING")
    print("="*60)
    
    for config_name, config in configs.items():
        print(f"\n{config_name.upper()} Configuration:")
        print(f"Config: {config}")
        print("-" * 30)
        
        for i, text in enumerate(test_texts[:2]):  # Test first 2 for brevity
            cleaned = clean_text(text, config)
            tokens = tokenize_text(text, config)
            
            print(f"Original {i+1}: {text}")
            print(f"Cleaned {i+1}:  {cleaned}")
            print(f"Tokens {i+1}:   {tokens}")
            print()

# Run preprocessing tests
test_preprocessing_pipeline()

In [ ]:
# Transformer-Compatible Preprocessing Functions
def prepare_transformer_inputs(texts: List[str], labels: List[int], 
                              tokenizer_name: str = 'bert-base-uncased',
                              max_length: int = 128, 
                              padding: str = 'max_length',
                              truncation: bool = True) -> Dict[str, Any]:
    """
    Prepare input data for transformer models using HuggingFace tokenizers.
    
    Args:
        texts: List of input texts
        labels: List of corresponding labels
        tokenizer_name: Name of the tokenizer to use
        max_length: Maximum sequence length
        padding: Padding strategy
        truncation: Whether to truncate long sequences
        
    Returns:
        Dictionary containing tokenized inputs and labels
    """
    try:
        from transformers import AutoTokenizer
        
        # Initialize tokenizer
        tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, 
                                                cache_dir='data/cache/transformers')
        
        # Tokenize texts
        print(f"Tokenizing {len(texts)} texts with {tokenizer_name}...")
        
        # Batch tokenization for efficiency
        encoding = tokenizer(
            texts,
            truncation=truncation,
            padding=padding,
            max_length=max_length,
            return_tensors='pt'
        )
        
        # Convert labels to tensor
        labels_tensor = torch.tensor(labels, dtype=torch.long)
        
        result = {
            'input_ids': encoding['input_ids'],
            'attention_mask': encoding['attention_mask'],
            'labels': labels_tensor,
            'tokenizer': tokenizer
        }
        
        print(f"✓ Tokenization complete. Shape: {encoding['input_ids'].shape}")
        
        return result
        
    except Exception as e:
        print(f"Error in transformer preprocessing: {e}")
        raise

# Preprocessing Ablation Study
def run_preprocessing_ablation(texts: List[str], labels: List[int], 
                             dataset_name: str, n_samples: int = 1000) -> Dict[str, Any]:
    """
    Run ablation study on preprocessing choices to quantify their impact.
    
    This provides evidence-based guidance for preprocessing decisions.
    """
    print(f"\nRUNNING PREPROCESSING ABLATION FOR {dataset_name}")
    print("="*60)
    
    # Sample data for faster ablation
    if len(texts) > n_samples:
        indices = np.random.choice(len(texts), n_samples, replace=False)
        sample_texts = [texts[i] for i in indices]
        sample_labels = [labels[i] for i in indices]
    else:
        sample_texts, sample_labels = texts, labels
    
    # Split for quick evaluation
    X_train, X_val, y_train, y_val = train_test_split(
        sample_texts, sample_labels, test_size=0.3, random_state=DEFAULT_SEED, 
        stratify=sample_labels
    )
    
    # Test different preprocessing configurations
    ablation_configs = {
        'baseline': PreprocessingConfig(lowercase=False, remove_punctuation=False, 
                                      remove_stopwords=False, lemmatize=False),
        'lowercase': PreprocessingConfig(lowercase=True, remove_punctuation=False, 
                                       remove_stopwords=False, lemmatize=False),
        'no_punct': PreprocessingConfig(lowercase=True, remove_punctuation=True, 
                                      remove_stopwords=False, lemmatize=False),
        'no_stopwords': PreprocessingConfig(lowercase=True, remove_punctuation=True, 
                                          remove_stopwords=True, lemmatize=False),
        'full': PreprocessingConfig(lowercase=True, remove_punctuation=True, 
                                  remove_stopwords=True, lemmatize=True)
    }
    
    ablation_results = {}
    
    for config_name, config in ablation_configs.items():
        try:
            # Create vectorizer with current config
            vectorizer = create_tfidf_vectorizer(
                ngram_range=(1, 1), 
                max_features=5000, 
                preprocessing_config=config
            )
            
            # Fit and transform
            start_time = time.perf_counter()
            X_train_vec = vectorizer.fit_transform(X_train)
            X_val_vec = vectorizer.transform(X_val)
            vectorize_time = time.perf_counter() - start_time
            
            # Quick classification test
            clf = MultinomialNB()
            clf.fit(X_train_vec, y_train)
            val_acc = clf.score(X_val_vec, y_val)
            
            # Store results
            ablation_results[config_name] = {
                'accuracy': val_acc,
                'vocab_size': len(vectorizer.vocabulary_),
                'vectorize_time': vectorize_time,
                'feature_density': X_train_vec.nnz / X_train_vec.shape[0]  # Avg features per sample
            }
            
            print(f"{config_name:12} | Acc: {val_acc:.3f} | Vocab: {len(vectorizer.vocabulary_):5d} | "
                  f"Time: {vectorize_time:.2f}s | Density: {X_train_vec.nnz / X_train_vec.shape[0]:.1f}")
            
        except Exception as e:
            print(f"{config_name:12} | ERROR: {e}")
            ablation_results[config_name] = {'error': str(e)}
    
    print("="*60)
    
    # Find best configuration
    valid_results = {k: v for k, v in ablation_results.items() if 'error' not in v}
    if valid_results:
        best_config = max(valid_results.keys(), key=lambda k: valid_results[k]['accuracy'])
        print(f"Best preprocessing configuration: {best_config} "
              f"(Accuracy: {valid_results[best_config]['accuracy']:.3f})")
    
    return ablation_results

# Run ablation studies for all datasets
print("Conducting preprocessing ablation studies...")

# Set random seed for consistent ablation
np.random.seed(DEFAULT_SEED)
random.seed(DEFAULT_SEED)

# Run ablations (using smaller sample sizes for efficiency during development)
ag_ablation = run_preprocessing_ablation(ag_train_texts, ag_train_labels, "AG_News", n_samples=500)
ng_ablation = run_preprocessing_ablation(ng_train_texts, ng_train_labels, "20_Newsgroups", n_samples=500) 
imdb_ablation = run_preprocessing_ablation(imdb_train_texts, imdb_train_labels, "IMDb", n_samples=500)

# 5. Baseline Classical Models (Detailed Implementation + Narrative)

## Classical Machine Learning: The Foundation of Text Classification

Before the deep learning revolution transformed NLP, classical machine learning approaches dominated text classification tasks. These methods, particularly Multinomial Naïve Bayes and Support Vector Machines with TF-IDF features, established the fundamental principles that continue to influence modern approaches.

### Theoretical Foundations:

**Multinomial Naïve Bayes (MNB)**: Based on Bayes' theorem with the "naïve" assumption of feature independence. Despite this strong assumption being violated in natural language, MNB often performs surprisingly well due to its robustness and the prevalence of discriminative features in text.

**Linear Support Vector Machines (LinearSVM)**: Implements the principle of structural risk minimization, finding the optimal hyperplane that maximizes the margin between classes. The linear kernel is particularly well-suited for high-dimensional sparse text features.

**TF-IDF Features**: Term Frequency-Inverse Document Frequency creates a vector space representation where each dimension represents a term's importance, balancing local term frequency with global discriminative power.

### Implementation Strategy:

Our implementation employs scikit-learn's robust pipeline infrastructure, enabling systematic hyperparameter optimization while maintaining clean separation of concerns between preprocessing, feature extraction, and classification.

### Performance Considerations:

Classical methods excel in computational efficiency, interpretability, and performance on smaller datasets. They serve as essential baselines and often remain competitive with more complex approaches, particularly when computational resources are constrained.